#Next Frame Prediction using Pix2PixHD

By Derrick Schultz

Forked repo and tutorial based on [JC Testud’s excellent repo and Medium](https://medium.com/@jctestud/video-generation-with-pix2pix-aed5b1b69f57) article.

## Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Check to see what GPU we’re assigned

In [7]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-12215951-5394-74ae-b203-d00f5e178325)


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Install libraries and dependencies



In [4]:
import os
if os.path.isdir("/content/drive/MyDrive/nfp-colab/pix2pixHD/"):
    %cd /content/drive/MyDrive/nfp-colab/pix2pixHD/
    # !git pull
    !pip install dominate
else:
    %cd /content/drive/MyDrive
    !mkdir nfp-colab
    %cd nfp-colab
    !git clone -b video https://github.com/dvschultz/pix2pixHD.git
    !pip install dominate
    %cd pix2pixHD/

/content/drive/MyDrive/nfp-colab/pix2pixHD


## Extract frames from video

Upload a video to either Colab or Google Drive.

* `-video` is the path to the video file
* `-name` should be a unique name (think of it like a project name)
* `-width` and `-height` **must** to be a multiple of 32
* `-p2pdir` leave as `.` unless you know it shouldn’t be that ;)


In [ ]:
!python extract_frames.py -video /content/Datasets/landscape.mp4 -name landscape -p2pdir . -width 1280 -height 736

creating the dataset structure
ffmpeg -v 16 -i /content/Datasets/landscape.mp4 -q:v 2 -vf "scale=iw*736/ih:736, crop=1280:736" /content/drive/My\ Drive/nfp-colab/pix2pixHD/datasets/landscape/train_frames/frame-%06d.jpg -hide_banner
extracting the frames


## Train your model

### Initial training

Note: if you have a large dataset, this may timeout initially (`ValueError: __len__() should return >= 0`). Give it a minute or two and run it again.

*   `--name` should be a unique name (think of it like a project name). For your sanity I recommend using the same `-name` as above.
*   `--dataroot` should point to your dataset. This will usually be in `./datasets/[your project name]`

 



In [ ]:
!python train_video.py --name landscape --dataroot ./datasets/landscape/ --save_epoch_freq 5 --continue_train

------------ Options -------------
batchSize: 1
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: True
data_type: 32
dataroot: ./datasets/landscape/
debug: False
display_freq: 100
display_winsize: 512
feat_num: 3
fineSize: 512
fp16: False
fps: 24.0
gpu_ids: [0]
heat_seeking_lvl: 0
input_nc: 3
instance_feat: False
isTrain: True
label_feat: False
label_nc: 35
lambda_feat: 10.0
loadSize: 1024
load_features: False
load_pretrain: 
local_rank: 0
lr: 0.0002
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: landscape
ndf: 64
nef: 16
netG: global
ngf: 64
niter: 100
niter_decay: 100
niter_fix_global: 0
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: False
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 3
phase: train
pool_size: 0
print_freq: 100
pstart: 1
pstop: 1
resize_or_crop: scale_width
save_epoch_freq: 5
save_

### Continue Training
To pick up from a previous training session run the same command you ran to start with and append `--continue_train` to the end of the command.

In [ ]:
!python train_video.py --name cuttlefish1 --dataroot ./datasets/cuttlefish1/ --save_epoch_freq 1 --continue_train

#Generating Videos

To generate a video from your completed model, run the `generate_video.py` script. I recommend keeping the `--name` and `--dataroot` arguments the same.

There are a number of additional arguments you’ll need to include in this command:


*   `--fps` frame rate for your video
*   `--how_many` how many frames you want to create (this + fps = video length)
*   `--which_epoch` which epoch you want to generate videos from (note: if you choose `133` but there’s no epoch 133 model file, this will throw an error)
*   `--start_from` by default your video will start predicting images from the 60th frame of your training set. You can pass in the path to a different frame to have it start from that frame

I recommend playing with both the `--which_epoch` and `--start_from` arguments as you can get dramatically different results by changing in the inputs here.





In [ ]:
!python generate_video.py --name cuttlefish1 --dataroot ./datasets/cuttlefish1/ --fps 24 --how_many 600 --which_epoch latest 

In [ ]:
import os

def processFolder(folder, epoch = 10, frameCount = 240, skipCount = 1):
  files = os.listdir(folder)

  count = 0
  for f in files:
    
    if (count % skipCount == 0):
      print(f)
      if epoch == 'latest':
        command = 'python generate_video.py --name glitch_white_circle --dataroot ./datasets/glitch-circle-white_dataset/ --fps 60 --how_many %i --which_epoch latest --start_from %s/%s' % ( frameCount, folder, f)
      else:
        command = 'python generate_video.py --name glitch_white_circle --dataroot ./datasets/glitch-circle-white_dataset/ --fps 24 --how_many %i --which_epoch %i --start_from %s/%s' % ( frameCount, epoch, folder, f)
      os.system(command)
    count += 1

processFolder('./datasets/fuck-white/train_frames',1,600,3050)